In [5]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 8.0 MB/s eta 0:00:00:00:0100:01


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import RobustScaler, StandardScaler, LabelEncoder, MinMaxScaler, MaxAbsScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
import xgboost as xgb
#from catboost import CatBoostRegressor
from numpy import mean
from numpy import std

In [ ]:
casas = pd.read_csv("kc_house_data.csv")

casas.drop(['id'],axis=1,inplace=True)
casas.drop(['date'],axis=1,inplace=True)


In [ ]:
def create_model(data, target, algo, scaler=None):
	if scaler != None:
		X = scaler.fit_transform(data)
	else:
		X = data
	y = target
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=10)
	model = algo
	model.fit(X_train, y_train)
	return(model, X_train, X_test, y_train, y_test)

casas = casas.drop_duplicates()

condition = (casas['sqft_living'] > casas['sqft_lot']) & (casas['floors'] == 1)
for index, row in casas[condition].iterrows():
    casas.at[index, 'sqft_living'] = row['sqft_living15']
    casas.at[index, 'sqft_lot'] = row['sqft_lot15']

condition = (casas['sqft_living15'] > casas['sqft_lot15']) & (casas['floors'] == 1)
for index, row in casas[condition].iterrows():
    casas.at[index, 'sqft_living15'] = row['sqft_living']
    casas.at[index, 'sqft_lot15'] = row['sqft_lot']

scaler = MinMaxScaler()
casas['bedrooms'] = scaler.fit_transform(casas[['bedrooms']])
casas['grade'] = scaler.fit_transform(casas[['grade']])
casas['bathrooms'] = scaler.fit_transform(casas[['bathrooms']])
casas['floors'] = scaler.fit_transform(casas[['floors']])
casas['view'] = scaler.fit_transform(casas[['view']])
casas['sqft_above'] = scaler.fit_transform(casas[['sqft_above']])
casas['yr_built'] = scaler.fit_transform(casas[['yr_built']])
casas['sqft_living'] = scaler.fit_transform(casas[['sqft_living']])
casas['sqft_lot'] = scaler.fit_transform(casas[['sqft_lot']])
casas['sqft_living15'] = scaler.fit_transform(casas[['sqft_living15']])
casas['sqft_lot15'] = scaler.fit_transform(casas[['sqft_lot15']])
casas['condition'] = scaler.fit_transform(casas[['condition']])
casas['sqft_basement'] = scaler.fit_transform(casas[['sqft_basement']])
casas['yr_renovated'] = scaler.fit_transform(casas[['yr_renovated']])
casas['zipcode'] = scaler.fit_transform(casas[['zipcode']])
casas['lat'] = scaler.fit_transform(casas[['lat']])
casas['long'] = scaler.fit_transform(casas[['long']])

# Especificar colunas de features (X) e coluna alvo (y)
features = casas.drop("price", axis=1)  # Remova a coluna alvo "price" das features
target = casas["price"]

# Testando diversos regressores
modelo, X_train, X_test, y_train, y_test = create_model(features, target, RandomForestRegressor(), scaler=None)
#modelo, X_train, X_test, y_train, y_test = create_model(features, target, LinearRegression(), scaler=None)
#modelo, X_train, X_test, y_train, y_test = create_model(features, target, SVR(), scaler=None)
#modelo, X_train, X_test, y_train, y_test = create_model(features, target, GradientBoostingRegressor(), scaler=None)
#modelo, X_train, X_test, y_train, y_test = create_model(features, target, KNeighborsRegressor(), scaler=None)
#modelo, X_train, X_test, y_train, y_test = create_model(features, target, Lasso(), scaler=None)
#modelo, X_train, X_test, y_train, y_test = create_model(features, target, Ridge(), scaler=None)
#modelo, X_train, X_test, y_train, y_test = create_model(features, target, ElasticNet(), scaler=None)

previsoes = modelo.predict(X_test)
mse = mean_squared_error(y_test, previsoes)
print(f'Mean Squared Error (MSE): {mse}')

y_pred = modelo.predict(X_test)
print(f'R2 (Test) : {r2_score(y_test, y_pred)}')
print(f'R2 (Training) : {r2_score(y_true=y_train, y_pred=modelo.predict(X_train))}')

Mean Squared Error (MSE): 17060347063.674498
R2 (Test) : 0.8720480558642185
R2 (Training) : 0.9814984396131553
